In [1]:

from projimmo.data import hello
hello()

hello toi


In [2]:
import sys
sys.path

['/home/cpoulard/code/CecilePoulard/04-Decision-Science/01-Project-Setup/data-context-and-setup',
 '/home/cpoulard/code/CecilePoulard/projimmo',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/cpoulard/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/cpoulard/.pyenv/versions/3.10.6/envs/projimmo/lib/python3.10/site-packages']

In [3]:
import projimmo

projimmo.__file__

'/home/cpoulard/code/CecilePoulard/projimmo/projimmo/__init__.py'

In [2]:
import os
import pandas as pd
from projimmo.params import *
#data=os.getenv('DATA_DIR')
print(DATA_DIR)

chunk = pd.read_csv(DATA_DIR, sep="|", nrows=CHUNK_SIZE, dtype=str)
chunk.head()
#'/home/cpoulard/projimmo/raw_data/valeursfoncieres-2023.txt'


raw_data/valeursfoncieres-2023.txt


,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,3,Dépendance,NaN,0,0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,1,2,Appartement,NaN,233,8,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,03/01/2023,Vente,...,NaN,0,1,Maison,NaN,64,3,S,NaN,988
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000001,05/01/2023,Vente,...,NaN,0,1,Maison,NaN,73,3,S,NaN,835
